In [ ]:
# 1) Instalar y cargar dataset Cats vs Dogs
# pip install tensorflow tensorflow-datasets -q

import tensorflow as tf
# install into the notebook kernel (use %pip so it installs to the kernel's env)
%pip install -q tensorflow-datasets

import tensorflow_datasets as tfds  # type: ignore
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

IMG_SIZE = 128
BATCH = 32
AUTOTUNE = tf.data.AUTOTUNE

(ds_train, ds_val), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:]'],
    with_info=True, as_supervised=True
)

def prep(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

train = ds_train.map(prep, num_parallel_calls=AUTOTUNE).shuffle(1000).batch(BATCH).prefetch(AUTOTUNE)
val   = ds_val.map(prep,   num_parallel_calls=AUTOTUNE).batch(BATCH).prefetch(AUTOTUNE)

# 2) Definir una CNN compacta
model = keras.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train, validation_data=val, epochs=3)  # sube epochs si tu GPU/CPU lo permite

***Visualizar filtros de la primera capa***